#  Training Model- Solution B - BiLSTM

In [ ]:
# Training Model Code - Solution B - BILSTM

# !cat /proc/cpuinfo | grep "model name" | uniq
# !nvidia-smi
# Please install these libraries below
# !pip install torch
# !pip install transformers
# !pip install tqdm
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn

# Use the A100 GPU to run this code

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import json
import os
import re
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AutoModel, BertModel
from sklearn.metrics import f1_score, accuracy_score
from itertools import product

# File path of dataset
train_file_path = '/content/drive/My Drive/nlu-lab/train.csv'
validation_file_path = '/content/drive/My Drive/nlu-lab/dev.csv'

# Load training and testing data
train_data = pd.read_csv(train_file_path)
val_data = pd.read_csv(validation_file_path)

# Check the data loading
print(train_data.head(2))
print(val_data.head(2))

Mounted at /content/drive
                                 Claim  \
0  We should introduce school vouchers   
1   We should legalize insider trading   

                                            Evidence  label  
0  Among the many educational reform efforts, suc...      0  
1  The U.S. Securities and Exchange Commission wa...      0  
                                 Claim  \
0  Democracy should be done away with.   
1       Polygamy should be made legal.   

                                            Evidence  label  
0  Amartya Sen, an Indian economist and Nobel lau...      0  
1  The Supreme Court's unanimous decision in Reyn...      1  


# Text Preprocessing

In [ ]:
# text preprocessing
def clean_text(text):
  text = re.sub(r"can't\b", "cannot", text)  # Replace can't with cannot
  text = re.sub(r"won't\b", "will not", text)  # Replace won't with will not
  text = re.sub(r"n't\b", " not", text)  # Replace n't with not
  text = re.sub(r"'re\b", " are", text)  # Replace 're with are
  text = re.sub(r"'m\b", " am", text)  # Replace 'm with am
  text = re.sub(r"'ve\b", " have", text)  # Replace 've with have
  text = re.sub(r"'ll\b", " will", text)  # Replace 'll with will
  text = re.sub(r"'d\b", " would", text)  # Replace 'd with would
  # Expand only pronoun + 's
  text = re.sub(r"\b(he|she|it|that|what|who|there|where|why|when)'s\b", r"\1 is", text, flags=re.IGNORECASE)
  # url
  text = re.sub(r'http\S+|www\S+|https\S+', '[URL]', text)
  # blank space
  text = re.sub(r"\s+", " ", text).strip()
  return text

train_data['combined_text'] = (train_data['Claim'] + " " + train_data['Evidence']).apply(clean_text)
val_data['combined_text'] = (val_data['Claim'] + " " + val_data['Evidence']).apply(clean_text)

# Check the cleaned data
print(train_data[['combined_text']].head())
print(val_data[['combined_text']].head())

                                       combined_text
0  We should introduce school vouchers Among the ...
1  We should legalize insider trading The U.S. Se...
2  We should subsidize investigative journalism T...
3  We should further exploit nuclear power a 2001...
4  We should ban whaling The US and several other...
                                       combined_text
0  Democracy should be done away with. Amartya Se...
1  Polygamy should be made legal. The Supreme Cou...
2  Hunting should be banned In total it is estima...
3  Television should be given up. Barbera mention...
4  Abortions ought to be prohibited. According to...


# BiLSTM Classifier Using BERT Word Embeddings

In [3]:
class BERT_LSTMClassifier(nn.Module):
  def __init__(self, hidden_size=128, num_classes=2, dropout=0.3, num_layers=2):
    super(BERT_LSTMClassifier, self).__init__()
    self.bert_embeddings = self._load_bert_embeddings('bert-base-uncased')

    self.lstm = nn.LSTM(
        input_size=768,
        hidden_size=hidden_size,
        num_layers=num_layers,
        batch_first=True,
        bidirectional=True
    )
    self.dropout = nn.Dropout(dropout)
    self.fc = nn.Linear(hidden_size * 2, num_classes)

  def _load_bert_embeddings(self, model_path):
    bert = AutoModel.from_pretrained(model_path)
    return bert.embeddings.word_embeddings

  def forward(self, input_ids, attention_mask):
    embeddings = self.bert_embeddings(input_ids)
    lstm_out, _ = self.lstm(embeddings)
    pooled_output = torch.mean(lstm_out, dim=1)

    out = self.dropout(pooled_output)
    logits = self.fc(out)
    return logits

# Training & Evaluation Function

In [ ]:
# Parameters
MAX_LEN = 128

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

#load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Dataset class
class EDDataset(Dataset):
  def __init__(self, texts, labels=None):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, index):
    text = self.texts[index]
    encoding = self.tokenizer(text, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='pt')
    item = {
        'input_ids': encoding['input_ids'].squeeze(0),
        'attention_mask': encoding['attention_mask'].squeeze(0)
    }
    if self.labels is not None:
      item['labels'] = torch.tensor(self.labels[index], dtype=torch.long)
    return item

# Function to train model, tqdm used for clear training process
def train_model(model, data_loader, optimizer, criterion, device, num_epochs, show_progress=False):
  model.train()
  for epoch in range(1, num_epochs + 1):
    total_loss = 0

    if show_progress:
      tqdm.write(f"Epoch {epoch}/{num_epochs}")

    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      optimizer.zero_grad()
      outputs = model(input_ids, attention_mask=attention_mask)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    tqdm.write(f"Epoch {epoch}: Avg Loss = {avg_loss:.4f}")

# Function to evaluate model
def evaluate_model(model, data_loader, device):
  model.eval()
  all_preds, all_labels = [], []
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask)
      preds = torch.argmax(outputs, dim=1)

      all_preds.extend(preds.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())

  acc = accuracy_score(all_labels, all_preds)
  f1 = f1_score(all_labels, all_preds)
  return acc, f1


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Model Training

In [ ]:
# param grid
param_grid = {
    'learning_rate': [2e-5, 5e-5],
    'batch_size': [16, 32],
    'epochs': [3, 5, 10],
    'dropout': [0.1, 0.3],
    'lstm_hidden_size': [128, 256],
    'num_lstm_layers': [1, 2],
}

# param combitions
param_combinations = list(product(
    param_grid['learning_rate'],
    param_grid['batch_size'],
    param_grid['epochs'],
    param_grid['dropout'],
    param_grid['lstm_hidden_size'],
    param_grid['num_lstm_layers'],
))

best_f1 = 0
best_params = {}

model_dir = '/content/drive/My Drive/nlu-lab/lstm_models'
os.makedirs(model_dir, exist_ok=True)

results = []

# training
for idx, (lr, bs, epochs, dropout, lstm_hidden_size, num_lstm_layers) in enumerate(tqdm(param_combinations, desc="Grid Search Progress")):
  tqdm.write(f"\n[{idx+1}/{len(param_combinations)}] Training with lr={lr}, bs={bs}, epochs={epochs}, dropout={dropout}, lstm_hidden_size={lstm_hidden_size}, num_lstm_layers={num_lstm_layers}")

  train_dataset = EDDataset(train_data['combined_text'].tolist(), train_data['label'].tolist())
  val_dataset = EDDataset(val_data['combined_text'].tolist(), val_data['label'].tolist())
  train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=bs)

  model = BERT_LSTMClassifier(hidden_size=lstm_hidden_size, num_layers=num_lstm_layers, num_classes=2, dropout=dropout).to(device)
  optimizer = AdamW(model.parameters(), lr=lr)
  criterion = nn.CrossEntropyLoss()

  train_model(model, train_loader, optimizer, criterion, device, epochs, show_progress=True)

  acc, f1 = evaluate_model(model, val_loader, device)
  tqdm.write(f"Validation Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")

  results.append({
      'lr': lr,
      'batch_size': bs,
      'epochs': epochs,
      'dropout': dropout,
      'lstm_hidden_size': lstm_hidden_size,
      'num_lstm_layers': num_lstm_layers,
      'acc': acc,
      'f1': f1
  })
  # Choose the best parameters according to F1 scores, then save the best model.
  if f1 > best_f1:
    best_f1 = f1
    best_params = {
        'learning_rate': lr,
        'batch_size': bs,
        'epochs': epochs,
        'dropout': dropout,
        'lstm_hidden_size': lstm_hidden_size,
        'num_lstm_layers': num_lstm_layers
    }

    # check path exists
    if not os.path.exists(model_dir):
      os.makedirs(model_dir)

    torch.save(model.state_dict(), os.path.join(model_dir, 'best_model_bilstm.pt'))
    tqdm.write("Best model saved.")

with open(os.path.join(model_dir, 'grid_search_results_bilstm.json'), 'w') as f:
  json.dump(results, f, indent=2)

print(f"\nBest Parameters: {best_params}, Best F1 Score: {best_f1:.4f}")


Grid Search Progress:   0%|          | 0/96 [00:00<?, ?it/s]


[1/96] Training with lr=2e-05, bs=16, epochs=3, dropout=0.1, lstm_hidden_size=128, num_lstm_layers=1


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Grid Search Progress:   0%|          | 0/96 [00:06<?, ?it/s]

...

Epoch 9: Avg Loss = 0.1470
Epoch 10/10


Grid Search Progress:  99%|█████████▉| 95/96 [5:08:57<04:29, 269.54s/it]

Epoch 10: Avg Loss = 0.1280


Grid Search Progress: 100%|██████████| 96/96 [5:09:04<00:00, 193.17s/it]

Validation Accuracy: 0.7771, F1 Score: 0.5959

Best Parameters: {'learning_rate': 2e-05, 'batch_size': 16, 'epochs': 5, 'dropout': 0.1, 'lstm_hidden_size': 256, 'num_lstm_layers': 2}, Best F1 Score: 0.6594


# Save JSON File containing best parameter setting.

In [ ]:
# Use independent code block to generate best parameter file in case of model training process being interrupted.
import json
import os

# path
model_dir = '/content/drive/My Drive/nlu-lab/lstm_models'
grid_search_path = os.path.join(model_dir, 'grid_search_results_bilstm.json')
best_param_path = os.path.join(model_dir, 'best_params_bilstm.json')

# load gird search and save best parameters
with open(grid_search_path, 'r') as f:
  results = json.load(f)

if results:
  best_result = max(results, key=lambda x: x['f1'])

  with open(best_param_path, 'w') as f:
    json.dump(best_result, f, indent=2)

  print(f"\nBest Parameters saved to: {best_param_path}")
  print(f"Best F1 Score: {best_result['f1']:.4f}")
  print(f"Parameters: {best_result}")
else:
  print("grid_search_results_bilstm.json is empty.")


Best Parameters saved to: /content/drive/My Drive/nlu-lab/lstm_models/best_params_bilstm.json
Best F1 Score: 0.6594
Parameters: {'lr': 2e-05, 'batch_size': 16, 'epochs': 5, 'dropout': 0.1, 'lstm_hidden_size': 256, 'num_lstm_layers': 2, 'acc': 0.7878839014512319, 'f1': 0.6594418856678407}


# Make sure the model file is being overwritten by last training process

In [7]:
# check model saved successfully or not
import os
from datetime import datetime

save_path = '/content/drive/My Drive/nlu-lab/lstm_models/best_model_bilstm.pt'

if os.path.exists(save_path):
  print(f"model exist: {save_path}")
  print("latest modify time:", datetime.fromtimestamp(os.path.getmtime(save_path)))
else:
  print(f"can't find model file, please check.")



model exist: /content/drive/My Drive/nlu-lab/lstm_models/best_model_bilstm.pt
latest modify time: 2025-04-10 02:09:54
